In [1]:
import data1
import torch, torch.utils, torch.utils.data
from unet import UNet
from tqdm.notebook import tqdm as tqdm
from torchsummary import summary
import matplotlib.pyplot as plt
import modeltools1
import datetime
import random
import torchvision
import os

device = torch.device("cuda")
weights_path = "weights-FocalLoss-30000epochs.dat"
out_dir = "out1"
if not os.path.exists(out_dir): os.mkdir(out_dir)
#weights_path = None

In [2]:
train_data_dir = "../01_image_segmentation1/01_train"
train_data_filename = "idx-train.txt" #should change to _orig?
dataset_train = data1.Dataset(train_data_dir, train_data_filename)

val_data_dir = "../01_image_segmentation1/02_test_clean"
val_data_filename = "idx-test.txt"
dataset_val = data1.Dataset(val_data_dir, val_data_filename)
dataset_val.eval()

dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size = 4, num_workers=1, 
    pin_memory=True, persistent_workers=True)

dataloader_val = torch.utils.data.DataLoader(dataset_train, batch_size = 4, num_workers=1, 
    pin_memory=True, persistent_workers=True)

Loading raw data: 0it [00:00, ?it/s]

Loading raw data: 0it [00:00, ?it/s]

AttributeError: 

In [ ]:
model = UNet(in_channels = 3, out_channels=3, n_blocks = 4).to(device)
#uncomment to check the model summary
summary(model, (3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             896
              ReLU-2         [-1, 32, 128, 128]               0
       BatchNorm2d-3         [-1, 32, 128, 128]              64
            Conv2d-4         [-1, 32, 128, 128]           9,248
              ReLU-5         [-1, 32, 128, 128]               0
       BatchNorm2d-6         [-1, 32, 128, 128]              64
         MaxPool2d-7           [-1, 32, 64, 64]               0
         DownBlock-8  [[-1, 32, 64, 64], [-1, 32, 128, 128]]               0
            Conv2d-9           [-1, 64, 64, 64]          18,496
             ReLU-10           [-1, 64, 64, 64]               0
      BatchNorm2d-11           [-1, 64, 64, 64]             128
           Conv2d-12           [-1, 64, 64, 64]          36,928
             ReLU-13           [-1, 64, 64, 64]               0
      BatchNorm2d-14      

In [ ]:
lr = 0.01
optim_params = model.parameters()
optim = torch.optim.Adam(optim_params, lr=lr)
# loss = torch.nn.CrossEntropyLoss()
loss = modeltools1.FocalLoss(reduction="mean")
scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma =0.9999)#gamma=0.997)

In [ ]:
losses = []
if ("weights_path" in locals() or "weights_path" in globals()) and weights_path:
    model.load_state_dict(torch.load(weights_path))
else:
    modeltools1.train_model(model, loss, optim, scheduler, dataloader_train,
        num_epochs = 1000, device = device) #30 000 takes one hour and suffices
    model.cpu()
    current_time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    torch.save(model.state_dict(), './weights'+current_time+".dat")

In [ ]:
# dataset_train.eval()
# image = dataset_train[0][0]
# label = dataset_train[0][1]
# label_masks = modeltools1.classes_to_masks(label)


# model.cuda()

# prediction = modeltools1.tiled_eval(model, image, 16, (128,128), (64,64), device, device)

In [ ]:
# pred_mask = modeltools1.logits_to_masks(prediction)
# masked = modeltools1.apply_mask(image, pred_mask, colors=modeltools1.DEFAULT_OVERLAY_COLORS, alpha=0.3)
# torchvision.io.write_png(masked, out_dir+"/test.png")
# score = modeltools1.F1_score(pred_mask, label_masks)

In [ ]:

dataset_train.eval()
total_score = modeltools1.eval_on_dataset(model, dataset_train, out_dir+"/train", 8,
    (128,128), (64,64), device, device, [])

print(f"Average F1 score on train set: {total_score}")

Images in dataset:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Score:  0.9896859028681583


  0%|          | 0/1 [00:00<?, ?it/s]

Score:  0.9756355753103113


  0%|          | 0/1 [00:00<?, ?it/s]

Score:  0.9884495924530853


  0%|          | 0/1 [00:00<?, ?it/s]

Score:  0.979255389288355
Average F1 score on train set: 0.9832566149799775


In [ ]:
total_score = modeltools1.eval_on_dataset(model, dataset_val, out_dir+"/val", 8,
    (128,128), (64,64), device, device, [])

print(f"Average F1 score on val set: {total_score}")

Images in dataset:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Score:  0.9868730022347889


  0%|          | 0/1 [00:00<?, ?it/s]

Score:  0.6543010512349633


  0%|          | 0/1 [00:00<?, ?it/s]

Score:  0.9848253238730399


  0%|          | 0/1 [00:00<?, ?it/s]

Score:  0.9862564513661655
Average F1 score on val set: 0.9030639571772393


In [ ]:
total_score

0.9030639571772393